In [15]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [16]:
from datasets import load_dataset

ds = load_dataset("dominguesm/alpaca-data-pt-br")["train"].select(range(100))  # só 100 exemplos p/ teste
ds = ds.train_test_split(test_size=0.1)
ds.save_to_disk("alpaca-pt-mini")


Saving the dataset (1/1 shards): 100%|██████████| 10/10 [00:00<00:00, 4498.88 examples/s]


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

# Modelo base
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cpu"  # desativa mapeamento automático
).to("cpu") 

# Configuração do LoRA
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config).to("cpu")

# Dataset pequeno em português
dataset = load_dataset("dominguesm/alpaca-data-pt-br")
dataset = dataset["train"].select(range(200))  # reduzido para testar em CPU
dataset = dataset.train_test_split(test_size=0.1)

# Tokenização
def tokenize(example):
    prompt = f"Instrução: {example['instruction']}\nResposta: {example['output']}"
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize)

# Treinamento
training_args = TrainingArguments(
    output_dir="./tinyllama-lora",
    per_device_train_batch_size=1,        # Evita estouro de memória na CPU
    gradient_accumulation_steps=8,        # Equivale a batch_size = 8
    num_train_epochs=5,                   # Mais épocas = melhor ajuste
    learning_rate=2e-5,                   # Aprendizado estável
    logging_steps=10,                  # Log a cada 10 passos
    use_cpu=True,                       # Força uso de CPU
    save_strategy="epoch",
    eval_strategy="epoch",
    warmup_steps=10,                      # Evita explosões no início
    weight_decay=0.01,                    # Regularização leve
    fp16=False,                           # FP16 não é suportado na CPU
    bf16=False,
    dataloader_num_workers=0,             # Evita múltiplos processos
    push_to_hub=False                     # Não envia para HF
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)
import torch
print("Usando CUDA?", torch.cuda.is_available())
print("Dispositivo do modelo:", next(model.parameters()).device)
trainer.train()


/tmp/ipykernel_10641/2992770869.py:55: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Usando CUDA? True
Dispositivo do modelo: cpu


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:

model = model.merge_and_unload()  # Incorpora os pesos LoRA no modelo base
model.save_pretrained("./tinyllama-lora/merged")
tokenizer.save_pretrained("./tinyllama-lora/merged")


('./tinyllama-lora/merged/tokenizer_config.json',
 './tinyllama-lora/merged/special_tokens_map.json',
 './tinyllama-lora/merged/tokenizer.json')